### TODO:
1. ellenőrizni megyéket: lett évről-évre eltérés? --> ha nincs nagy eltérés, hozzuk egy oszlopra
2. adatokat aggregálni megye illetve gazdálkodási forma és ágazat kategória szerint
3. feature_importance check
4. cross-validation
5. number of trees to use: plot the training: num trees on x and error on y axis

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk

In [15]:
from sklearn.model_selection import train_test_split

target_column = 'target_reg'

train_df = pd.read_csv('./dataset/train.csv')
evaluation_df = pd.read_csv('./dataset/test.csv')


# Clean data, reduce features

In [3]:
train_df.dropna(inplace=True)

In [53]:
COLUMN_GROUPS = [
    ['MEGYE_2014', 'MEGYE_2015', 'MEGYE_2016'], 
    ['SZEKHELY_IR_SZAM_2014', 'SZEKHELY_IR_SZAM_2015', 'SZEKHELY_IR_SZAM_2016'],
    ['GAZDALKODASI_FORMA_2014_fo_kategoria', 'GAZDALKODASI_FORMA_2015_fo_kategoria', 'GAZDALKODASI_FORMA_2016_fo_kategoria'],
    ['GAZDALKODASI_FORMA_2014_1_alkategoria', 'GAZDALKODASI_FORMA_2015_1_alkategoria', 'GAZDALKODASI_FORMA_2016_1_alkategoria'],
    ['GAZDALKODASI_FORMA_2014_2_alkategoria', 'GAZDALKODASI_FORMA_2015_2_alkategoria', 'GAZDALKODASI_FORMA_2016_2_alkategoria'],
    ['KKV_BESOROLAS_2014_fo_kategoria', 'KKV_BESOROLAS_2015_fo_kategoria', 'KKV_BESOROLAS_2016_fo_kategoria'],
    ['KKV_BESOROLAS_2014_1_alkategoria', 'KKV_BESOROLAS_2015_1_alkategoria', 'KKV_BESOROLAS_2016_1_alkategoria'],
    ['KKV_BESOROLAS_2014_2_alkategoria', 'KKV_BESOROLAS_2015_2_alkategoria', 'KKV_BESOROLAS_2016_2_alkategoria'],
    ['agazat_2014_fo_kategoria', 'agazat_2015_fo_kategoria', 'agazat_2016_fo_kategoria'],
    ['agazat_2014_1_alkategoria', 'agazat_2015_1_alkategoria', 'agazat_2016_1_alkategoria'],
    ['agazat_2014_2_alkategoria', 'agazat_2015_2_alkategoria', 'agazat_2016_2_alkategoria'],
    ['agazat_2014_3_alkategoria', 'agazat_2015_3_alkategoria', 'agazat_2016_3_alkategoria'],
]

In [59]:
from itertools import combinations


def check_if_difference_in_columns(df, columns):
    for pair in combinations(columns, 2):
        column_a, column_b = pair[0], pair[1]
        differing_number_of_rows = df.loc[df[column_a] != df[column_b]].shape[0]
        if differing_number_of_rows > 0:
            print(
                f'different number of rows: {differing_number_of_rows}',
                f'found in columns: {column_a}, {column_b}',
                sep=' '
            )
            return True

    return False

def check_column_groups_for_difference(df, column_groups):
    for group in column_groups:
        check_if_difference_in_columns(df, group)




In [60]:
check_column_groups_for_difference(df=train_df, column_groups=COLUMN_GROUPS)

different number of rows: 24 found in columns: KKV_BESOROLAS_2014_fo_kategoria, KKV_BESOROLAS_2015_fo_kategoria
different number of rows: 24 found in columns: KKV_BESOROLAS_2014_1_alkategoria, KKV_BESOROLAS_2015_1_alkategoria
different number of rows: 24 found in columns: KKV_BESOROLAS_2014_2_alkategoria, KKV_BESOROLAS_2015_2_alkategoria
different number of rows: 501 found in columns: agazat_2014_fo_kategoria, agazat_2015_fo_kategoria
different number of rows: 604 found in columns: agazat_2014_1_alkategoria, agazat_2015_1_alkategoria
different number of rows: 618 found in columns: agazat_2014_2_alkategoria, agazat_2015_2_alkategoria
different number of rows: 1969 found in columns: agazat_2014_3_alkategoria, agazat_2015_3_alkategoria


In [61]:
check_column_groups_for_difference(df=evaluation_df, column_groups=COLUMN_GROUPS)

different number of rows: 16 found in columns: KKV_BESOROLAS_2014_fo_kategoria, KKV_BESOROLAS_2015_fo_kategoria
different number of rows: 16 found in columns: KKV_BESOROLAS_2014_1_alkategoria, KKV_BESOROLAS_2015_1_alkategoria
different number of rows: 16 found in columns: KKV_BESOROLAS_2014_2_alkategoria, KKV_BESOROLAS_2015_2_alkategoria
different number of rows: 471 found in columns: agazat_2014_fo_kategoria, agazat_2015_fo_kategoria
different number of rows: 584 found in columns: agazat_2014_1_alkategoria, agazat_2015_1_alkategoria
different number of rows: 607 found in columns: agazat_2014_2_alkategoria, agazat_2015_2_alkategoria
different number of rows: 1589 found in columns: agazat_2014_3_alkategoria, agazat_2015_3_alkategoria


from the 3 megye columns 1 is enough, since there was no change in any of the rows

In [31]:
train_df_reduced_features = train_df.copy()

train_df_reduced_features.drop(['MEGYE_2015', 'MEGYE_2016'], axis='columns', inplace=True)
train_df_reduced_features.rename(columns={'MEGYE_2014': 'megye'}, inplace=True)

In [32]:
print(train_df_reduced_features.loc[train_df_reduced_features['SZEKHELY_IR_SZAM_2014'] != train_df_reduced_features['SZEKHELY_IR_SZAM_2015']].shape)
print(train_df_reduced_features.loc[train_df_reduced_features['SZEKHELY_IR_SZAM_2015'] != train_df_reduced_features['SZEKHELY_IR_SZAM_2016']].shape)
print(train_df_reduced_features.loc[train_df_reduced_features['SZEKHELY_IR_SZAM_2014'] != train_df_reduced_features['SZEKHELY_IR_SZAM_2016']].shape)


(0, 130)
(0, 130)
(0, 130)


szekhely columns don't change either, we can keep one

In [33]:
train_df_reduced_features.drop(['SZEKHELY_IR_SZAM_2014', 'SZEKHELY_IR_SZAM_2015'], axis='columns', inplace=True)
train_df_reduced_features.rename(columns={'SZEKHELY_IR_SZAM_2016': 'iranyito_szam'}, inplace=True)

### target_reg min miért negatív? létezik negatív export? + plottal meg lehetne vizsgálni

In [ ]:
print(train_df.describe())

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from utils import columns_to_onehotencode

encoder = OneHotEncoder(handle_unknown='ignore')

encoder_df = pd.DataFrame(encoder.fit_transform(train_df[[*columns_to_onehotencode]]).toarray())
train_df.reset_index(inplace=True)
encoder_df.reset_index(inplace=True)
train_df = pd.concat([train_df, encoder_df], axis='columns')
print(train_df.shape)
print(train_df.isna().sum().sum())

# Train

In [ ]:
target_df = train_df[target_column]
train_df.drop([target_column, *columns_to_onehotencode], axis='columns', inplace=True)

train_df.columns = train_df.columns.map(str)

x_train, x_test, y_train, y_test = train_test_split(train_df, target_df, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=50, random_state=42)
rf.fit(x_train,y_train)
predictions = rf.predict(x_test)

# Evaluate

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
mse = mean_squared_error(predictions, y_test)
mae = mean_absolute_error(predictions, y_test)
print(f'mean squared error: {mse}')
print(f'mean absolute error: {mae}')